In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import time
import warnings
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn import datasets, linear_model
from numpy import newaxis
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers import BatchNormalization
from keras.models import Sequential
from keras import regularizers
import keras.backend as K
import statsmodels.formula.api as smf
from matplotlib import pyplot as plt
from keras.callbacks import ModelCheckpoint

plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 10)

# prevent tensorflow from allocating the entire GPU memory at once
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Hide messy TensorFlow warnings
#warnings.filterwarnings("ignore") #Hide messy Numpy warnings

%autosave 60

os.getcwd()

/usr/local/lib/python2.7/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)
Using TensorFlow backend.


Autosaving every 60 seconds


'/home/inonpe/jupyter/deep_learning_project_norrecampus/old_notebooks'

In [2]:
def load_data(filename, num_lags, bootstrap_size):
    assert bootstrap_size > num_lags
    
    # read data from file
    f = open(filename)
    series = []
    removed_seasonality = [] 
    removed_std = []
    missings = []
    for line in f:
        splt = line.split(",")
        series.append(float(splt[0]))
        removed_seasonality.append(float(splt[1]))
        removed_std.append(float(splt[2]))
        missings.append(int(splt[3]))
    series = np.array(series)
    removed_seasonality = np.array(removed_seasonality)
    removed_std = np.array(removed_std)
    missings = np.array(missings)
    f.close()

    # generate lags
    X = []
    for i in range(bootstrap_size, len(series)):
        X.append(series[i-num_lags:i])
    X = np.array(X)
    
    y = series[bootstrap_size:]
    removed_seasonality = removed_seasonality[bootstrap_size:]
    removed_std = removed_std[bootstrap_size:]
    missings = missings[bootstrap_size:]
    assert X.shape[0] == y.shape[0]

    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    return X, y, removed_seasonality, removed_std, missings

In [3]:
PLACE_IDS_ORDERED_FROM_NORTH_TO_SOUTH = [
    'ChIJZaR1M1hSUkYRxP0WkwYYy_k',
    'ChIJj1RhMlhSUkYRxwx00g4P0QE',
    'ChIJuYkcKlhSUkYRFPCipW5rTvU',
    'ChIJP6TdhFdSUkYRdrsWKXZMAs8',
    'ChIJf9Y9sFdSUkYRmaDEJhCweGc',
    'ChIJozaGTFZSUkYRNtWl2AGUPkI',
    'ChIJ4QuVTlZSUkYRRDRPcHdYULQ',
    'ChIJBTt5RlZSUkYR_SyA8BgiwaM',
    'ChIJbcDEbFZSUkYRcnQFsIj5j5U'
]

In [4]:
# read data from different places
NUM_LAGS = 50
bootstrap_size = 12*24*1
ids = []
removed_seasonality_all = []
removed_std_all = []
missings_all = []
X_all = []
y_all = []
for fname in map(lambda place_id: place_id + '.csv', PLACE_IDS_ORDERED_FROM_NORTH_TO_SOUTH):
    print "reading data for place id:", fname[-6:-4]
    X, y, removed_seasonality, removed_std, missings = load_data('norrecampus/'+fname, NUM_LAGS, bootstrap_size)
    ids.append(fname[-6:-4])
    X_all.append(X)
    y_all.append(y)
    removed_seasonality_all.append(removed_seasonality)
    removed_std_all.append(removed_std)
    missings_all.append(missings)
X_all = np.array(X_all)
y_all = np.array(y_all)
print X_all.shape
print y_all.shape
n_places = len(ids)
n_instances = X_all.shape[1]
print "n_instances:", n_instances
print "n_places:", n_places
print "n_lags:", NUM_LAGS

reading data for place id: _k
reading data for place id: QE
reading data for place id: vU
reading data for place id: s8
reading data for place id: Gc
reading data for place id: kI
reading data for place id: LQ
reading data for place id: aM
reading data for place id: 5U
(9, 51552, 50, 1)
(9, 51552)
n_instances: 51552
n_places: 9
n_lags: 50


In [5]:
# reshape data
X = np.swapaxes(X_all, 0, 1)
X = np.swapaxes(X, 1, 2)
X = X[:,:,:,:,np.newaxis]
print X.shape
y = np.swapaxes(y_all, 0, 1)
y = y[:,np.newaxis,:,np.newaxis,np.newaxis]
print y.shape

STEPS_AHEAD = 1
X = X[:,:NUM_LAGS-STEPS_AHEAD+1,:,:]
print X.shape

(51552, 50, 9, 1, 1)
(51552, 1, 9, 1, 1)
(51552, 50, 9, 1, 1)


In [6]:
n_train = 12*24*89
n_test = n_train + 12*24*90
X_train = X[:n_train,:]
y_train = y[:n_train]
X_test = X[n_train:n_test,:]
y_test = y[n_train:n_test]
print X_train.shape
print y_train.shape
print X_test.shape
print y_test.shape

(25632, 50, 9, 1, 1)
(25632, 1, 9, 1, 1)
(25920, 50, 9, 1, 1)
(25920, 1, 9, 1, 1)


In [7]:
def compute_error(trues, predicted):
    corr = np.corrcoef(predicted, trues)[0,1]
    mae = np.mean(np.abs(predicted - trues))
    mse = np.mean((predicted - trues)**2)
    rae = np.sum(np.abs(predicted - trues)) / np.sum(np.abs(trues - np.mean(trues)))
    rmse = np.sqrt(np.mean((predicted - trues)**2))
    r2 = max(0, 1 - np.sum((trues-predicted)**2) / np.sum((trues - np.mean(trues))**2))
    return corr, mae, mse, rae, rmse, r2

In [8]:
def compute_error_filtered(trues, predicted, filt):
    trues = trues[filt]
    predicted = predicted[filt]
    corr = np.corrcoef(predicted, trues)[0,1]
    mae = np.mean(np.abs(predicted - trues))
    mse = np.mean((predicted - trues)**2)
    rae = np.sum(np.abs(predicted - trues)) / np.sum(np.abs(trues - np.mean(trues)))
    rmse = np.sqrt(np.mean((predicted - trues)**2))
    r2 = max(0, 1 - np.sum((trues-predicted)**2) / np.sum((trues - np.mean(trues))**2))
    return corr, mae, mse, rae, rmse, r2

In [9]:
def eval_quantiles(lower, upper, trues, preds):
    N = len(trues)
    icp = 1.0*np.sum((trues>lower) & (trues<upper)) / N
    diffs = np.maximum(0, upper-lower)
    mil = np.sum(diffs) / N
    rmil = 0.0
    for i in xrange(N):
        if trues[i] != preds[i]:
            rmil += diffs[i] / (np.abs(trues[i]-preds[i]))
    rmil = rmil / N
    clc = np.exp(-rmil*(icp-0.95))
    return icp, mil, rmil, clc

In [10]:
# place id of interest
ix = 7
print ids[ix]

aM


In [11]:
# true values we want to predict
y_true = y_test[:,0,ix,0,0] * removed_std_all[ix][n_train:n_test] + removed_seasonality_all[ix][n_train:n_test]

# naive baseline
preds_naive = y[n_train-STEPS_AHEAD:n_test-STEPS_AHEAD,0,ix,0,0] * removed_std_all[ix][n_train:n_test] + removed_seasonality_all[ix][n_train:n_test]
corr, mae_naive, mse, rae, rmse_naive, r2_naive = compute_error(y_true, preds_naive)
print "NAIVE: %.3f   %.3f   %.3f" % (mae_naive,rmse_naive,r2_naive)

# linear regression
regr = linear_model.LinearRegression()
regr.fit(X_train[:,:,ix,0,0], y_train[:,0,ix,0,0])
preds_lr = regr.predict(X_test[:,:,ix,0,0]) * removed_std_all[ix][n_train:n_test] + removed_seasonality_all[ix][n_train:n_test]

corr, mae_lr, mse, rae, rmse_lr, r2_lr = compute_error(y_true, preds_lr)
print "LR:    %.3f   %.3f   %.3f" % (mae_lr,rmse_lr,r2_lr)

NAIVE: 2.808   4.920   0.724
LR:    2.287   4.245   0.794


In [12]:
def tilted_loss(q,y,f):
    e = (y-f)
    # The term inside k.mean is a one line simplification of the first equation
    return K.mean(q*e + K.clip(-e, K.epsilon(), np.inf), axis=-1)

In [19]:
def build_model(loss="mse", num_outs=1):
    model = Sequential()

    #model.add(LSTM(layers[1], input_shape=(None, layers[0]), return_sequences=True))
    model.add(ConvLSTM2D(filters=20, kernel_size=(3, 1),
                   input_shape=(None, 9, 1, 1),
                   padding='same', return_sequences=False))
#     model.add(Dropout(0.2))

#     model.add(ConvLSTM2D(filters=20, kernel_size=(3, 3),
#                        padding='same', return_sequences=False))
    
    #model.add(LSTM(layers[2], return_sequences=False))
#     model.add(Dropout(0.2))
    
    #model.add(Dense(units=1, kernel_regularizer=regularizers.l2(0.00001)))
    model.add(Dense(units=num_outs))
    model.add(Activation("linear"))

    model.compile(loss=loss, optimizer="rmsprop")
    
    print(model.summary())
    
    return model

In [20]:
print X_train.shape
print X_train[:,:,:,0,0].transpose([0,2,1]).shape
print y_train.shape

(25632, 50, 9, 1, 1)
(25632, 9, 50)
(25632, 1, 9, 1, 1)


In [21]:
model = build_model(num_outs=1)

# checkpoint best model
checkpoint = ModelCheckpoint('./tmp/nonstacked_ordered_places_convlstm_mean.best.hdf5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

model.fit(
    X_train,
    y_train[:,:,:,:,0].swapaxes(1,2),
    batch_size=512,
    epochs=100,
    validation_split=0.2,
    callbacks=[checkpoint],
    verbose=2)  

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_3 (ConvLSTM2D)  (None, 9, 1, 20)          5120      
_________________________________________________________________
dense_3 (Dense)              (None, 9, 1, 1)           21        
_________________________________________________________________
activation_3 (Activation)    (None, 9, 1, 1)           0         
Total params: 5,141
Trainable params: 5,141
Non-trainable params: 0
_________________________________________________________________
None
Train on 20505 samples, validate on 5127 samples
Epoch 1/100
Epoch 00000: val_loss improved from inf to 0.50244, saving model to ./tmp/ordered_places_convlstm_mean.best.hdf5
5s - loss: 0.7847 - val_loss: 0.5024
Epoch 2/100
Epoch 00001: val_loss improved from 0.50244 to 0.34021, saving model to ./tmp/ordered_places_convlstm_mean.best.hdf5
5s - loss: 0.4890 - val_loss: 0.3402
Epoch 3/100
Epoch 0000

Epoch 00071: val_loss did not improve
5s - loss: 0.2568 - val_loss: 0.2320
Epoch 73/100
Epoch 00072: val_loss did not improve
5s - loss: 0.2569 - val_loss: 0.2313
Epoch 74/100
Epoch 00073: val_loss did not improve
5s - loss: 0.2568 - val_loss: 0.2275
Epoch 75/100
Epoch 00074: val_loss did not improve
5s - loss: 0.2564 - val_loss: 0.2285
Epoch 76/100
Epoch 00075: val_loss did not improve
5s - loss: 0.2564 - val_loss: 0.2278
Epoch 77/100
Epoch 00076: val_loss did not improve
5s - loss: 0.2561 - val_loss: 0.2349
Epoch 78/100
Epoch 00077: val_loss did not improve
5s - loss: 0.2561 - val_loss: 0.2289
Epoch 79/100
Epoch 00078: val_loss did not improve
5s - loss: 0.2561 - val_loss: 0.2273
Epoch 80/100
Epoch 00079: val_loss did not improve
5s - loss: 0.2559 - val_loss: 0.2292
Epoch 81/100
Epoch 00080: val_loss did not improve
5s - loss: 0.2556 - val_loss: 0.2317
Epoch 82/100
Epoch 00081: val_loss did not improve
5s - loss: 0.2557 - val_loss: 0.2332
Epoch 83/100
Epoch 00082: val_loss did not im

In [22]:
# load weights
model.load_weights('./tmp/nonstacked_ordered_places_convlstm_mean.best.hdf5')

# make predictions
predictions = model.predict(X_test)
print predictions.shape

(25920, 9, 1, 1)


In [27]:
predictions_trended = predictions.squeeze()[:, ix] * removed_std_all[ix][n_train:n_test] + removed_seasonality_all[ix][n_train:n_test]
print predictions_trended
print y_true

def performance(model_name, y_true, preds):
    corr, mae, mse, rae, rmse, r2 = compute_error(y_true, preds)
    print "%s:\tMAE %.3f ; RMSE %.3f ; R2 %.3f" % (model_name, mae , rmse, r2)
    
performance('NN', y_true, predictions_trended)

[ 26.97888096  39.07551036  29.41581836 ...,  32.27583918  30.72750961
  35.95252135]
[ 34.73226457  33.06830204  33.16504252 ...,  30.12590295  30.56989625
  31.9929323 ]
NN:	MAE 1.965 ; RMSE 3.926 ; R2 0.824
